In [4]:
import torch

In [24]:
words2=open('first-names.txt','r').read().splitlines()
words2=[word.lower() for word in words2]

In [27]:
words2[:10]

['aaren',
 'aarika',
 'abagael',
 'abagail',
 'abbe',
 'abbey',
 'abbi',
 'abbie',
 'abby',
 'abbye']

In [6]:
chars = sorted(list(set(''.join(words2))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [31]:
#basic bigram structure of one word
for w in words2[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)
# '.' indicates start and end 

. a
a a
a r
r e
e n
n .


In [32]:
xs, ys = [], []

for w in words2:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

In [33]:
xs

tensor([ 0,  3,  3,  ...,  3, 16,  3])

In [34]:
ys

tensor([ 3,  3, 20,  ..., 16,  3,  0])

In [35]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=29).float()
xenc

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [36]:
w = torch.randn((29,29))
xenc @ w

tensor([[ 0.1967, -2.7180, -0.1984,  ..., -0.3107,  0.8231, -0.3658],
        [ 0.0974, -1.4651, -0.5163,  ...,  0.1253, -0.1510, -0.5046],
        [ 0.0974, -1.4651, -0.5163,  ...,  0.1253, -0.1510, -0.5046],
        ...,
        [ 0.0974, -1.4651, -0.5163,  ...,  0.1253, -0.1510, -0.5046],
        [-0.8981,  0.4444,  0.3490,  ..., -1.2383,  0.7975, -0.3611],
        [ 0.0974, -1.4651, -0.5163,  ...,  0.1253, -0.1510, -0.5046]])

In [37]:
g = torch.Generator().manual_seed(2147483647)
w = torch.randn((29, 29), generator=g, requires_grad=True)

In [38]:
torch.arange(5)

tensor([0, 1, 2, 3, 4])

In [39]:
for k in range(50):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=29).float()
  logits = xenc @ w 
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(w**2).mean()
  print(loss.item())
  
  # backward pass
  w.grad = None 
  loss.backward()
  
  # update
  w.data += -50 * w.grad

3.846654176712036
3.3511712551116943
3.1088364124298096
2.949463367462158
2.8380074501037598
2.757619857788086
2.697125196456909
2.6501410007476807
2.612696409225464
2.582167625427246
2.5567691326141357
2.535273313522339
2.516833543777466
2.500858783721924
2.4869205951690674
2.4746897220611572
2.4638991355895996
2.4543285369873047
2.4457924365997314
2.438136339187622
2.43123197555542
2.42497181892395
2.4192678928375244
2.414046287536621
2.4092447757720947
2.4048118591308594
2.4007041454315186
2.3968846797943115
2.393322229385376
2.3899896144866943
2.38686466217041
2.3839266300201416
2.381159782409668
2.3785476684570312
2.376077890396118
2.37373948097229
2.371521234512329
2.369415044784546
2.3674123287200928
2.365506172180176
2.3636903762817383
2.3619582653045654
2.3603055477142334
2.358727216720581
2.3572187423706055
2.355776071548462
2.354395627975464
2.353074550628662
2.351808786392212
2.3505959510803223


In [40]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix= 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=29).float()
        logits = xenc @ w # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True) # probabilities for next character
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

h.
mibrianiaierin.
adory.
ally.
stalermarie.
de.
risa.
elybsthane.
tana.
b-ganipugia.
